In [25]:
import pandas as pd
import numpy as np
import json
import os
from typing import Dict, List, Union, Any
import re
import csv

In [ ]:
# Convert Excel file to CSV format
# excel_path = "address_full_0712.xlsx"
# csv_path = "address_full_0712.csv"

# df = pd.read_excel(excel_path)
# df.to_csv(csv_path, index=False, encoding="utf-8-sig")  

In [3]:
# Concatenate multiple JSON files into one
input_folder = "C:\\Users\\ply58\\OneDrive\\Tài liệu\\PythonML\\ML_git\\Python_ML\\BaitapDS108\\Lab 5\\LLMs Data"
output_file = "full_output.json"

json_files = [f for f in os.listdir(input_folder) if f.endswith(".json")]
json_files.sort()  

all_data = []

for file_name in json_files:
    file_path = os.path.join(input_folder, file_name)
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line.strip())
            all_data.append(obj)

with open(output_file, "w", encoding="utf-8") as f_out:
    for item in all_data:
        f_out.write(json.dumps(item, ensure_ascii=False) + "\n")

In [ ]:
# Reset ID in the concatenated JSON file
def reset_ids(input_file: str, output_file: str):
    with open(input_file, "r", encoding="utf-8") as f:
        lines = [json.loads(line) for line in f]

    for new_id, entry in enumerate(lines):
        entry["Id"] = new_id

    with open(output_file, "w", encoding="utf-8") as f:
        for entry in lines:
            f.write(json.dumps(entry, ensure_ascii=False) + "\n")

reset_ids("full_output.json", "full_output_by_llama.json")

Đến đây, em sẽ dùng address data ở: https://github.com/ThangLeQuoc/vietnamese-provinces-database để lọc các thành phố/tỉnh và các cấp đơn vị thấp hơn, trước tiên em sẽ xử lý bộ data trên, lọc và trích xuất các tên tỉnh/thành phố và quận/huyện

In [ ]:
# def count_administrative_units(data: Union[Dict, List[Dict]]) -> Dict[str, Dict[str, int]]:
#     counts = {
#         'provinces': 0,
#         'districts': 0,
#         'wards': 0,
#         'total': 0
#     }
    
#     detailed_counts = {
#         # Cấp tỉnh/thành phố
#         'provinces': 0,
#         'municipalities': 0,
        
#         # Cấp huyện/quận
#         'districts': 0,
#         'municipal_cities': 0,
#         'urban_districts': 0,
#         'district_level_towns': 0,
#         'provincial_districts': 0,
        
#         # Cấp xã/phường
#         'wards': 0,
#         'communes': 0,
#         'commune_level_towns': 0
#     }
    
#     def count_units(units: Union[Dict, List[Dict]], level: str) -> None:
#         nonlocal counts, detailed_counts
  
#         if isinstance(units, dict):
#             units = [units]
#         elif not isinstance(units, list):
#             return
            
#         for unit in units:
#             counts['total'] += 1
            
#             if level == 'province':
#                 counts['provinces'] += 1
                
#                 # Phân loại cấp tỉnh
#                 unit_type = unit.get('AdministrativeUnitShortName', '').lower()
#                 if 'thành phố' in unit_type:
#                     detailed_counts['municipalities'] += 1
#                 else:
#                     detailed_counts['provinces'] += 1
                
#                 # district 
#                 if 'District' in unit and isinstance(unit['District'], list) and len(unit['District']) > 0:
#                     count_units(unit['District'], 'district')
                    
#             elif level == 'district':
#                 counts['districts'] += 1
                
#                 # Phân loại cấp huyện/quận
#                 unit_type = unit.get('AdministrativeUnitShortName', '').lower()
#                 if 'quận' in unit_type:
#                     detailed_counts['urban_districts'] += 1
#                 elif 'thành phố' in unit_type:
#                     detailed_counts['municipal_cities'] += 1
#                 elif 'thị xã' in unit_type:
#                     detailed_counts['district_level_towns'] += 1
#                 elif 'huyện' in unit_type:
#                     detailed_counts['districts'] += 1
#                 else:
#                     detailed_counts['provincial_districts'] += 1
                
#                 # ward 
#                 if 'Ward' in unit and isinstance(unit['Ward'], list) and len(unit['Ward']) > 0:
#                     count_units(unit['Ward'], 'ward')
                    
#             elif level == 'ward':
#                 counts['wards'] += 1
                
#                 # Phân loại cấp xã/phường
#                 unit_type = unit.get('AdministrativeUnitShortName', '').lower()
#                 if 'phường' in unit_type:
#                     detailed_counts['wards'] += 1
#                 elif 'xã' in unit_type:
#                     detailed_counts['communes'] += 1
#                 elif 'thị trấn' in unit_type:
#                     detailed_counts['commune_level_towns'] += 1
    
#     #  cấp tỉnh
#     count_units(data, 'province')
    
#     return {
#         'summary': counts,
#         'detailed': detailed_counts
#     }

# def process_json_file(json_data: Union[str, Dict, List]) -> Union[Dict, None]:
#     if isinstance(json_data, str):
#         data = json.loads(json_data)
#     else:
#         data = json_data
    
#     # Đếm các đơn vị
#     result = count_administrative_units(data)
#     return result

# def load_json_from_file(file_path: str) -> Union[Dict, None]:
#     with open(file_path, 'r', encoding='utf-8') as file:
#         data = json.load(file)
#         return process_json_file(data)

# def display_results(result: Dict) -> None:
#     print('HỐNG KÊ TỔNG QUAN')
#     print(f"Tổng số đơn vị hành chính: {result['summary']['total']}")
#     print(f"- Cấp tỉnh/thành phố: {result['summary']['provinces']}")
#     print(f"- Cấp huyện/quận: {result['summary']['districts']}")
#     print(f"- Cấp xã/phường: {result['summary']['wards']}")
    
#     print('\nTHỐNG KÊ CHI TIẾT')
#     print('Cấp tỉnh/thành phố:')
#     print(f"- Tỉnh: {result['detailed']['provinces']}")
#     print(f"- Thành phố trực thuộc TW: {result['detailed']['municipalities']}")
    
#     print('\nCấp huyện/quận:')
#     print(f"- Huyện: {result['detailed']['districts']}")
#     print(f"- Quận: {result['detailed']['urban_districts']}")
#     print(f"- Thành phố thuộc tỉnh: {result['detailed']['municipal_cities']}")
#     print(f"- Thị xã: {result['detailed']['district_level_towns']}")
#     print(f"- Đơn vị hành chính khác cấp huyện: {result['detailed']['provincial_districts']}")
    
#     print('\nCấp xã/phường:')
#     print(f"- Phường: {result['detailed']['wards']}")
#     print(f"- Xã: {result['detailed']['communes']}")
#     print(f"- Thị trấn: {result['detailed']['commune_level_towns']}")

# def export_results_to_json(result: Dict, output_file: str) -> None:
#     with open(output_file, 'w', encoding='utf-8') as file:
#         json.dump(result, file, ensure_ascii=False, indent=2)

# if __name__ == "__main__":

#     file_path = "full_json_generated_data_vn_units.json"
#     result = load_json_from_file(file_path)
    
#     if result:
#         display_results(result)
        
#         output_file = "admin_units_count_result.json"
#         export_results_to_json(result, output_file)

HỐNG KÊ TỔNG QUAN
Tổng số đơn vị hành chính: 10794
- Cấp tỉnh/thành phố: 63
- Cấp huyện/quận: 696
- Cấp xã/phường: 10035

THỐNG KÊ CHI TIẾT
Cấp tỉnh/thành phố:
- Tỉnh: 57
- Thành phố trực thuộc TW: 6

Cấp huyện/quận:
- Huyện: 508
- Quận: 49
- Thành phố thuộc tỉnh: 87
- Thị xã: 52
- Đơn vị hành chính khác cấp huyện: 0

Cấp xã/phường:
- Phường: 1726
- Xã: 7692
- Thị trấn: 617


Xóa bỏ các đặc trưng không cần trong bài này của bộ data của nhóm tác giả có link github của data: https://github.com/ThangLeQuoc/vietnamese-provinces-database

In [3]:
def filter_unit_data(data):

    if isinstance(data, list):
        filtered_list = []
        for item in data:
            filtered_item = filter_unit_data(item)
            if filtered_item: 
                filtered_list.append(filtered_item)
        return filtered_list
    elif isinstance(data, dict):
        filtered_dict = {}
        if "Type" in data:
            filtered_dict["Type"] = data["Type"]
        if "Name" in data:
            filtered_dict["Name"] = data["Name"]
        if "FullName" in data:
            filtered_dict["FullName"] = data["FullName"]

        if "District" in data and isinstance(data["District"], list):
            filtered_districts = filter_unit_data(data["District"])
            if filtered_districts:
                filtered_dict["District"] = filtered_districts
        
        if "Ward" in data and isinstance(data["Ward"], list):
            filtered_wards = filter_unit_data(data["Ward"])
            if filtered_wards:
                filtered_dict["Ward"] = filtered_wards
        
        return filtered_dict
    return data 

input_file_path = "full_json_generated_data_vn_units.json"
output_file_path = "filtered_vn_units.json"

with open(input_file_path, "r", encoding="utf-8") as f:
    original_data = json.load(f)

filtered_data = filter_unit_data(original_data)

with open(output_file_path, "w", encoding="utf-8") as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=2)

#### Province

Vì những addresses có tên thành phố/tên tỉnh thì các đơn vị hành chính nhỏ hơn nó có thể xác định và verify dễ hơn do tính duy nhất, những addresses mà không có đơn vị hành chính là thành phố và tên tỉnh thì vẫn giữ nguyên vì không có đủ bằng chứng 

Tạo file "provinces.json" gồm các tên tỉnh/thành phố

In [4]:
def filter_provinces_and_municipalities(input_file: str, output_file: str):
    provinces_data = []
    
    with open(input_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    if isinstance(data, list):
        for item in data:
            if isinstance(item, dict) and "Type" in item:
                if item["Type"] == "province" or item["Type"] == "municipality":
                    provinces_data.append(item["Name"]) 
        
        with open(output_file, "w", encoding="utf-8") as f_out:
            json.dump(provinces_data, f_out, ensure_ascii=False, indent=2)

input_json_file = "filtered_vn_units.json"
output_json_file = "provinces.json"
filter_provinces_and_municipalities(input_json_file, output_json_file)

Tạo file "addresses_modification.json" để thực hiện xử lý các câu đã lọc

In [ ]:
def find_provinces_in_sentences(provinces_file, sentences_file, output_addresses_with_ids_file):
    with open(provinces_file, "r", encoding="utf-8") as f:
        province_names = json.load(f)

    province_names_lower = [name.lower() for name in province_names]
    special_cities = ["hà nội", "hồ chí minh", "đà nẵng", "cần thơ", "huế", "hải phòng"]

    addresses_for_modification = []

    with open(sentences_file, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line.strip())
            if "Sentence" not in data or not isinstance(data["Sentence"], str):
                continue

            sentence = data["Sentence"]
            sentence_lower = sentence.lower()

            parts = sentence_lower.rsplit(",", 1)
            if len(parts) < 2:
                continue

            after_comma = parts[1].strip()
            normalized = after_comma.replace("tỉnh", "").replace("thành phố", "").strip()

            if normalized not in province_names_lower:
                continue  

            found_wrong_match = False
            for prov in province_names_lower:
                valid_forms = [
                    prov,
                    f"tỉnh {prov}",
                    f"thành phố {prov}"
                ]

                wrong_matches = re.findall(r"\btỉnh\s+[^\s,]+(?:\s+[^\s,]+)*\b", sentence_lower)
                for match in wrong_matches:
                    if not any(match.strip() == valid for valid in valid_forms):
                        found_wrong_match = True
                        break
                if found_wrong_match:
                    break

            if found_wrong_match:
                continue  

            addresses_for_modification.append(data)

    with open(output_addresses_with_ids_file, "w", encoding="utf-8") as f_ids_out:
        json.dump(addresses_for_modification, f_ids_out, ensure_ascii=False, indent=2)

        
provinces_json_file = "provinces.json"
full_output_json_file = "full_output_by_llama.json"
output_addresses_with_ids_file = "addresses_modification.json"

find_provinces_in_sentences(provinces_json_file, full_output_json_file, output_addresses_with_ids_file)

Thêm các tiền tố như Tỉnh và Thành phố cho các câu đã được lọc và lưu vào file "addresses_with_prefix.json"

In [14]:
def add_province_prefix_if_missing(addresses_file, provinces_file, output_file):
    # Danh sách 6 thành phố trực thuộc trung ương 
    special_cities = ["hà nội", "hồ chí minh", "đà nẵng", "cần thơ", "huế", "hải phòng"]

    # Provinces file
    with open(provinces_file, "r", encoding="utf-8") as f:
        provinces = json.load(f)
    provinces_lower = [p.lower() for p in provinces]

    # addresses file
    with open(addresses_file, "r", encoding="utf-8") as f:
        addresses = json.load(f)

    updated_addresses = []
    for item in addresses:
        sentence = item.get("Sentence", "")
        sentence_lower = sentence.lower()
        modified = False

        for province in provinces_lower:
            if province in sentence_lower:
                # Check if the province is already have prefix
                if f"tỉnh {province}" in sentence_lower or f"thành phố {province}" in sentence_lower:
                    break 
                else:
                    # Add prefix if missing
                    if province in special_cities:
                        prefix = "Thành phố "
                    else:
                        prefix = "Tỉnh "

                    index = sentence_lower.find(province)
                    original_text = sentence[index: index + len(province)]
                    sentence = sentence[:index] + prefix + original_text + sentence[index + len(province):]
                    item["Sentence"] = sentence
                    modified = True
                    break

        updated_addresses.append(item)

    # Saving
    with open(output_file, "w", encoding="utf-8") as f_out:
        json.dump(updated_addresses, f_out, ensure_ascii=False, indent=2)

addresses_file="addresses_modification.json"
provinces_file="provinces.json"
output_file="addresses_with_prefix.json"
add_province_prefix_if_missing(addresses_file, provinces_file, output_file)

#### District

Lọc ra tên các cấp quận/huyện/thị xã/thành phố (thuộc tỉnh) và lưu vào "districts.json"

In [15]:
def extract_district_names(input_file: str, output_file: str):

    valid_types = {
        "district",
        "urban district",
        "district-level town",
        "provincial city"
    }

    district_entries = []

    # Input file
    with open(input_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    for province in data:
        province_name = province.get("Name", "")       
        for district in province.get("District", []):
            district_type = district.get("Type")
            name = district.get("Name")
            full_name = district.get("FullName")

            if district_type in valid_types and name and full_name:
                district_entries.append({
                    "Province": province_name,  
                    "Name": name,
                    "FullName": full_name
                })

    # Drop duplicates by Name
    seen = set()
    unique_districts = []
    for entry in district_entries:
        key = (entry["Name"], entry["Province"])
        if key not in seen:
            seen.add(key)
            unique_districts.append(entry)

    # Sort by Province, Name
    unique_districts.sort(key=lambda x: (x["Province"], x["Name"]))

    # Saving
    with open(output_file, "w", encoding="utf-8") as f_out:
        json.dump(unique_districts, f_out, ensure_ascii=False, indent=2)

input_file = "filtered_vn_units.json"
output_file = "districts.json"
extract_district_names(input_file, output_file)

Với "districs.json", sửa tiền tố của cấp District

In [16]:
def normalize_province_name(name: str):
    name = name.lower().strip()
    name = name.replace("tỉnh", "").replace("thành phố", "").strip()
    return name

def replace_district_names(address_file: str, districts_file: str, output_file: str, field: str = "Sentence"):
    # Load dữ liệu
    with open(address_file, "r", encoding="utf-8") as f:
        addresses = json.load(f)

    with open(districts_file, "r", encoding="utf-8") as f:
        districts = json.load(f)

    prefix_keywords = {"quận", "huyện", "thành phố", "thị xã"}

    district_dict = {}
    for d in districts:
        name_lower = d["Name"].lower()
        district_dict.setdefault(name_lower, []).append(d)

    modified_addresses = []
    for addr_obj in addresses:
        text = addr_obj.get(field, "")
        parts = [p.strip() for p in text.split(",")]

        if len(parts) < 2:
            modified_addresses.append(addr_obj)
            continue

        district_candidate = parts[-2]
        province_candidate = parts[-1]

        has_prefix = any(district_candidate.lower().startswith(prefix) for prefix in prefix_keywords)
        if has_prefix:
            modified_addresses.append(addr_obj)
            continue

        name_lower = district_candidate.lower()
        if name_lower in district_dict:
            matched = False
            for district_entry in district_dict[name_lower]:
                if normalize_province_name(province_candidate) == normalize_province_name(district_entry["Province"]):
                    parts[-2] = district_entry["FullName"]
                    matched = True
                    break

        addr_obj[field] = ", ".join(parts)
        modified_addresses.append(addr_obj)

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(modified_addresses, f, ensure_ascii=False, indent=2)

addresses_file = "addresses_with_prefix.json"
districts_file = "districts.json"
output_file = "addresses_with_prefix_districts.json"
replace_district_names(addresses_file, districts_file, output_file)

#### Ward

Lọc ra các cấp Xã/Phường/Thị trấn và lưu vào wards.json

In [ ]:
def extract_ward_names(input_file: str, output_file: str):

    valid_types = {
        "ward",                # Phường
        "commune",             # Xã
        "commune-level town"   # Thị trấn
    }

    ward_entries = []

    # Input file
    with open(input_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    for province in data:
        province_name = province.get("Name")
        for district in province.get("District", []):
            district_name = district.get("Name")
            for ward in district.get("Ward", []):
                ward_type = ward.get("Type")
                name = ward.get("Name")
                full_name = ward.get("FullName")

                if ward_type in valid_types and name and full_name:
                    ward_entries.append({
                        "Name": name,
                        "FullName": full_name,
                        "Province": province_name,
                        "District": district_name
                    })

    # Drop duplicates by Name, Province, District
    seen = set()
    unique_wards = []
    for entry in ward_entries:
        key = (entry["Name"], entry["Province"], entry["District"])
        if key not in seen:
            seen.add(key)
            unique_wards.append(entry)

    # Sort by Name 
    unique_wards.sort(key=lambda x: (x["Name"], x["Province"], x["District"]))

    # Saving
    with open(output_file, "w", encoding="utf-8") as f_out:
        json.dump(unique_wards, f_out, ensure_ascii=False, indent=2)

input_file = "filtered_vn_units.json"
output_file = "wards.json"
extract_ward_names(input_file, output_file)

In [18]:
def replace_ward_names(address_file: str, wards_file: str, output_file: str, field: str = "Sentence"):
    # Address file
    with open(address_file, "r", encoding="utf-8") as f:
        addresses = json.load(f)
    # Ward file
    with open(wards_file, "r", encoding="utf-8") as f:
        wards = json.load(f)
    # Sort wards by length of Name in descending order
    sorted_wards = sorted(wards, key=lambda w: len(w["Name"]), reverse=True)

    prefix_keywords = ["phường", "xã", "thị trấn"]
    modified_addresses = []

    # Replace ward names in addresses
    for addr_obj in addresses:
        original_text = addr_obj.get(field, "")
        text = original_text
        for ward in sorted_wards:
            name = ward["Name"]
            full_name = ward["FullName"]
            district = ward["District"].lower()
            province = ward["Province"].lower()

            # If the full name is already in the text -> skip 
            if re.search(re.escape(full_name), text, flags=re.IGNORECASE):
                continue

            # Find the ward name in the text
            pattern = r'(?<!\w)' + re.escape(name) + r'(?!\w)'
            match = re.search(pattern, text, flags=re.IGNORECASE)

            # If found, check the context before and after the match
            if match:
                before_text = text[max(0, match.start() - 20):match.start()].lower()
                if any(keyword in before_text.split()[-3:] for keyword in prefix_keywords):
                    continue

                after_text = text[match.end():].lower()
                has_district = district in after_text
                has_province = province in after_text

                if has_district and has_province:
                    text = re.sub(pattern, full_name, text, flags=re.IGNORECASE)
                    break  

        addr_obj[field] = text
        modified_addresses.append(addr_obj)

    # Saving
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(modified_addresses, f, ensure_ascii=False, indent=2)

address_file="addresses_with_prefix_districts.json"
wards_file="wards.json"
output_file="addresses_with_prefix_wards.json"
replace_ward_names(address_file, wards_file,output_file)

#### Result json

In [19]:
# full_output_by_llama.json
with open("full_output_by_llama.json", "r", encoding="utf-8") as f:
    content = f.read().strip()
    if content.startswith("["):
        full_data = json.loads(content)
    else:
        full_data = [json.loads(line) for line in content.splitlines() if line.strip()]

# addresses_with_prefix_wards.json
with open("addresses_with_prefix_wards.json", "r", encoding="utf-8") as f:
    ward_content = f.read().strip()
    if ward_content.startswith("["):
        ward_data = json.loads(ward_content)
    else:
        ward_data = [json.loads(line) for line in ward_content.splitlines() if line.strip()]

# Create a dictionary to map Id to Sentence from ward_data
ward_dict = {entry["Id"]: entry["Sentence"] for entry in ward_data if "Id" in entry and "Sentence" in entry}

# Replace "Sentence" in full_data with corresponding values from ward_dict
for item in full_data:
    if "Id" in item and item["Id"] in ward_dict:
        item["Sentence"] = ward_dict[item["Id"]]

# result.json
with open("result.json", "w", encoding="utf-8") as f:
    json.dump(full_data, f, ensure_ascii=False, indent=2)

Id

In [22]:
def Add_id(input_file, output_file, fields=None):
    with open(input_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    output_data = []

    if isinstance(data, list) and all(isinstance(item, str) for item in data):
        for i, name in enumerate(data, 1):
            output_data.append({
                "Id": i,
                "Name": name
            })
    elif isinstance(data, list) and all(isinstance(item, dict) for item in data):
        for i, item in enumerate(data, 1):
            entry = {field: item[field] for field in fields if field in item}
            entry["Id"] = i
            output_data.append(entry)
    else:
        raise ValueError(f"Không nhận diện được cấu trúc trong file {input_file}")

    with open(output_file, "w", encoding="utf-8") as f_out:
        json.dump(output_data, f_out, ensure_ascii=False, indent=2)

Add_id("provinces.json", "provinces_id.json")
Add_id("districts.json", "districts_id.json", fields=["Name", "Province"])
Add_id("wards.json", "wards_id.json", fields=["Name", "District", "Province"])

In [24]:
with open("provinces_id.json", "r", encoding="utf-8") as f:
    provinces = json.load(f)

with open("districts_id.json", "r", encoding="utf-8") as f:
    districts = json.load(f)

with open("wards_id.json", "r", encoding="utf-8") as f:
    wards = json.load(f)

with open("result.json", "r", encoding="utf-8") as f:
    results = json.load(f)

# Create maps for provinces, districts, and wards
province_map = {p["Name"].lower(): {"Id": p["Id"], "Name": p["Name"]} for p in provinces}

district_map = {}
for d in districts:
    province = d["Province"]
    if province not in district_map:
        district_map[province] = []
    district_map[province].append(d)

ward_map = {}
for w in wards:
    key = (w["Province"], w["District"])
    if key not in ward_map:
        ward_map[key] = []
    ward_map[key].append(w)

def normalize(text):
    return text.lower().strip()

def add_id_to_sentence(entry):
    sentence = entry["Sentence"]
    parts = [part.strip() for part in sentence.split(",")]
    if len(parts) < 2:
        return entry

    # Take the last part as the province name
    last = normalize(parts[-1])
    province_name = None
    for name in province_map:
        if name in last:
            province_name = province_map[name]["Name"]
            entry["city_id"] = province_map[name]["Id"]
            break

    if not province_name:
        return entry

    # Check districts
    if len(parts) >= 3:
        district_candidate = normalize(parts[-2])
        for d in district_map.get(province_name, []):
            if normalize(d["Name"]) in district_candidate:
                entry["district_id"] = d["Id"]
                district_name = d["Name"]

                # Check wards
                if len(parts) >= 4:
                    ward_candidate = normalize(parts[-3])
                    for w in ward_map.get((province_name, district_name), []):
                        if normalize(w["Name"]) in ward_candidate:
                            entry["ward_id"] = w["Id"]
                            break
                break

    return entry

# ID
updated_results = [add_id_to_sentence(entry) for entry in results]

# Saving
with open("result_id.json", "w", encoding="utf-8") as f:
    json.dump(updated_results, f, ensure_ascii=False, indent=2)

In [30]:
with open("result_id.json", "r", encoding="utf-8-sig") as f:
    data = json.load(f)

with open("result_id.csv", "w", newline="", encoding="utf-8-sig") as csvfile:
    fieldnames = ["id", "full_address", "city_id", "district_id"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for entry in data:
        writer.writerow({
            "id": entry.get("Id", ""),
            "full_address": entry.get("Sentence", ""),
            "city_id": entry.get("city_id", ""),
            "district_id": entry.get("district_id", ""),

        })